In [2]:
import requests
from bs4 import BeautifulSoup as bs

In [45]:
def get_rss_links(url):
    # Get rss links from a page (rss_link)
    url_page = requests.get(url).text
    
    bs_page = bs(url_page, "lxml")
    
    # x.link *should* work, but doesn't. Don't ask me why.
    article_links = map(lambda x: x.guid.text.strip(), bs_page.findAll("item"))

    return article_links

In [179]:
def get_article_text_from_link(url):
    # Get text from an article
    url_page = requests.get(url).text
    
    bs_page = bs(url_page, "lxml")
    
    if "dailymail" in url:
        article_main = bs_page.findAll("div", {"itemprop": "articleBody"})[0]
    
        article_content = map(lambda x: x.text.strip(), 
                         article_main.findAll("p", 
                                             {"class":
                                              "mol-para-with-font"}))
        
        article = " ".join(article_content)
        
    elif "theguardian" in url:
        article = bs_page.findAll("div", {"class": 
                                          lambda x: x and "content__article-body" 
                                          in x.split()})
        if article:
            article = article[0].text.strip()
        
        
    return article 

In [181]:
list(get_articles_from_rss("https://www.theguardian.com/science/rss"))

['The first and last supermoon of the year will rise above the horizon in the east on Sunday and loom larger and brighter than normal as it climbs in the night sky until it sets the next morning.\nKnown in the Farmer’s Almanac as the cold moon, the long night moon, and the moon before yule, the event comes as the December full moon coincides with the body’s close approach to Earth, making it appear 7% larger and 15% brighter than average.\n\n\n\n\nSupermoon science: November 2016 moon biggest and brightest in 60 years\n\n\n\n\n\n\n \n\n\nRead more\n\n\n\n\n\n\nThe moon’s orbit is not perfectly circular and its distance from Earth varies from around 222,000 miles to more than 250,000 miles over the course of a month. A full moon arises when the Earth sits directly between the sun and the moon.\nThe supermoon will rise from around 4.15pm GMT in Britain, but will be nearest to Earth shortly before 9am on Monday morning. When close to the horizon, the moon may take on a reddish hue, becaus

In [173]:
def get_articles_from_rss(rss_url):
    # Get articles from an rss page (rss_link)
    rss_links = get_rss_links(rss_url)
    article_texts = map(get_article_text_from_link, rss_links)
    
    return article_texts

In [138]:
def get_rss_sections(feeds_page):
    # Get all RSS sections from main, overall feed page (all_rss)
    url_page = requests.get(feeds_page).text
    bs_page = bs(url_page, "lxml")

    h3s = bs_page.findAll("h3", {"class": "wocc"})[:-1]
    
    h3_items = {}
    
    for item in h3s:
        section_title = item.text
        
        ul =  item.find_next("ul", {"class": "rss-group"})
        if not ul:
            continue
            
        li = ul.findAll("li")
        rss_items = {}
        for l in li:
            title = l.span.text
            link = l.find("span", {"class": "rss-btn rss"}).find('a')['href']
            rss_items[title] = link
            
        h3_items[section_title] = rss_items
        
        
    return h3_items

In [142]:
f = list(get_articles_from_rss(rss_link))

In [143]:
len(f)

78

In [85]:
url = "http://www.dailymail.co.uk/news/index.rss"
article_url = "http://www.dailymail.co.uk/news/article-5136989/Harvey-Weinstein-rape-case-no-longer-pursued-DA.html?ITO=1490&ns_mchannel=rss&ns_campaign=1490"
rss_link = "http://www.dailymail.co.uk/news/index.rss"
all_rss = "http://www.dailymail.co.uk/home/article-2684527/RSS-Feeds.html"